# Excercise on Autoencoder and Variational Autoencoder

Aim of this excercise is to implement an Autoencoder (AE) and Variational Autoencoder (VAE) similar to the tutorial.

For this excercise we want to focus on the Sea level pressure (SLP) in the North Atlantic which highly affects the climate over Europe. 

You will adapt the AE architecture of the tutorial to use Convolutional Neural Networks (CNN) and compare their performance. Furthermore, you will compare the latent space of the AE, VAE and PCA. Ideally you will identify the so called Northern Atlantic Oscillation (NAO), but more later ....